
# Datasets
Datasets는 Hugging Face에서 제공하는 데이터셋 처리 라이브러리입니다.

### 주요 특징
- **메모리 효율성**: Apache Arrow 기반으로 대용량 데이터도 효율적으로 처리
- **빠른 처리 속도**: 최적화된 데이터 로딩 및 처리
- **다양한 형식 지원**: 텍스트, 이미지, 오디오, 비디오 등
- **스트리밍 지원**: 큰 데이터셋을 메모리에 모두 로드하지 않고 처리 가능
- **Hub 연동**: Hugging Face Hub의 수천 개 데이터셋에 쉽게 접근

### 설치 방법
```bash
pip install datasets
```


In [1]:
%pip install datasets==4.0.0 transformers==4.56.1 torch==2.8.0

In [5]:
import datasets

from datasets import load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names
from datasets import Dataset, DatasetDict, IterableDataset
import pandas as pd
import numpy as np

print(f"Datasets 버전: {datasets.__version__}")

Datasets 버전: 4.0.0


## 2. 허브에서 데이터셋 로드하기

Hugging Face Hub에는 NLP, Computer Vision, Audio 등 다양한 분야의 데이터셋이 수천 개 제공됩니다.


### 2.1 데이터셋 정보 확인하기

데이터셋을 다운로드하기 전에 `load_dataset_builder()`를 사용해 정보를 미리 확인할 수 있습니다.


In [8]:
# 데이터셋 정보 확인하기
ds_builder = load_dataset_builder("imdb")

print("=== 데이터셋 설명 ===")
print(ds_builder.info.description[:300] + "...")

=== 데이터셋 설명 ===
...


In [9]:
print("=== 데이터셋 특성(Features) ===")
print(ds_builder.info.features)

=== 데이터셋 특성(Features) ===
{'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}


In [10]:
print("=== 데이터셋 크기 ===")
print(f"총 크기: {ds_builder.info.dataset_size}")
print(f"다운로드 크기: {ds_builder.info.download_size}")


=== 데이터셋 크기 ===
총 크기: 133202802
다운로드 크기: 83446840


### 2.2 스플릿(Split) 확인하기

대부분의 데이터셋은 train, validation, test로 나뉘어 있습니다.


In [11]:
# 데이터셋의 스플릿 확인
splits = get_dataset_split_names("imdb")
print("사용 가능한 스플릿:", splits)

사용 가능한 스플릿: ['train', 'test', 'unsupervised']


In [12]:
# 특정 스플릿만 로드
train_dataset = load_dataset("imdb", split="train")
print(f"\n훈련 데이터셋: {train_dataset}")
print(f"데이터 개수: {len(train_dataset)}")


훈련 데이터셋: Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})
데이터 개수: 25000


In [13]:
# 첫 번째 샘플 확인
print("\n=== 첫 번째 샘플 ===")
print(f"텍스트: {train_dataset[0]['text'][:200]}...")
print(f"라벨: {train_dataset[0]['label']}")



=== 첫 번째 샘플 ===
텍스트: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ev...
라벨: 0


In [ ]:
# 전체 데이터셋 로드 (모든 스플릿)
full_dataset = load_dataset("imdb")
print("전체 데이터셋:")
print(full_dataset)
print()

# 각 스플릿별 데이터 개수 확인
for split_name, split_data in full_dataset.items():
    print(f"{split_name}: {len(split_data):,}개")


### 2.3 구성(Configuration) 확인하기

일부 데이터셋은 여러 하위 데이터셋(구성)을 포함합니다. 예: 다국어 데이터셋


In [15]:
configs = get_dataset_config_names("PolyAI/minds14")
print("PolyAI/minds14 데이터셋의 구성들:")
print(configs[:10])  # 처음 10개만 출력

PolyAI/minds14 데이터셋의 구성들:
['all', 'cs-CZ', 'de-DE', 'en-AU', 'en-GB', 'en-US', 'es-ES', 'fr-FR', 'it-IT', 'ko-KR']


In [16]:
# 특정 언어 구성 로드
minds_en = load_dataset("PolyAI/minds14", "en-US", split="train[:100]")  # 처음 100개만
print(f"\n영어(en-US) 구성: {minds_en}")


영어(en-US) 구성: Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 100
})


In [18]:
print(f"첫 번째 샘플: {minds_en['transcription'][0]}")

첫 번째 샘플: I would like to set up a joint account with my partner


## 3. 데이터셋 생성하기

자신만의 데이터로 데이터셋을 생성하는 방법을 알아봅시다.



### 3.1 파일 기반 빌더

CSV, JSON, Parquet, TXT 등 다양한 파일 형식을 지원합니다.

In [19]:
# 샘플 CSV 데이터 생성
import csv
import tempfile
import os

# 임시 CSV 파일 생성
temp_dir = tempfile.mkdtemp()
csv_file = os.path.join(temp_dir, "sample_data.csv")

In [20]:
# 샘플 데이터 작성
sample_data = [
    ["text", "label"],
    ["이 영화는 정말 재미있어요!", 1],
    ["별로였어요. 추천하지 않습니다.", 0],
    ["훌륭한 연출과 연기였습니다.", 1],
    ["지루하고 재미없었어요.", 0],
    ["최고의 영화 중 하나입니다!", 1]
]

In [21]:
with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(sample_data)

print(f"CSV 파일 생성: {csv_file}")

CSV 파일 생성: /tmp/tmpq4imftub/sample_data.csv


In [22]:
# CSV에서 데이터셋 생성
csv_dataset = load_dataset("csv", data_files=csv_file)
print("CSV 데이터셋:")
print(csv_dataset)
print("\n첫 번째 샘플:")
print(csv_dataset["train"][0])


Generating train split: 0 examples [00:00, ? examples/s]

CSV 데이터셋:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

첫 번째 샘플:
{'text': '이 영화는 정말 재미있어요!', 'label': 1}


In [23]:
# JSON 파일에서 데이터셋 생성
import json

json_file = os.path.join(temp_dir, "sample_data.json")

In [24]:
# JSON 데이터 생성 (JSONL 형식)
json_data = [
    {"text": "멋진 영화였어요!", "label": 1, "rating": 4.5},
    {"text": "시간 낭비였습니다.", "label": 0, "rating": 2.0},
    {"text": "강력 추천합니다!", "label": 1, "rating": 5.0}
]

with open(json_file, 'w', encoding='utf-8') as f:
    for item in json_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"JSON 파일 생성: {json_file}")

JSON 파일 생성: /tmp/tmpq4imftub/sample_data.json


In [25]:
# JSON에서 데이터셋 생성
json_dataset = load_dataset("json", data_files=json_file)
print("JSON 데이터셋:")
print(json_dataset)
print("\n샘플:")
for i in range(len(json_dataset["train"])):
    print(json_dataset["train"][i])


Generating train split: 0 examples [00:00, ? examples/s]

JSON 데이터셋:
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'rating'],
        num_rows: 3
    })
})

샘플:
{'text': '멋진 영화였어요!', 'label': 1, 'rating': 4.5}
{'text': '시간 낭비였습니다.', 'label': 0, 'rating': 2.0}
{'text': '강력 추천합니다!', 'label': 1, 'rating': 5.0}


### 3.2 Python 딕셔너리에서 데이터셋 생성

메모리에 있는 Python 데이터로부터 직접 데이터셋을 생성할 수 있습니다.


In [26]:
# from_dict() 방법
data_dict = {
    "pokemon": ["피카츄", "파이리", "꼬부기", "이상해씨"],
    "type": ["전기", "불", "물", "풀"],
    "power": [90, 85, 80, 82]
}

dict_dataset = Dataset.from_dict(data_dict)
print("딕셔너리에서 생성한 데이터셋:")
print(dict_dataset)
print("\n샘플:")
for i in range(len(dict_dataset)):
    print(dict_dataset[i])


딕셔너리에서 생성한 데이터셋:
Dataset({
    features: ['pokemon', 'type', 'power'],
    num_rows: 4
})

샘플:
{'pokemon': '피카츄', 'type': '전기', 'power': 90}
{'pokemon': '파이리', 'type': '불', 'power': 85}
{'pokemon': '꼬부기', 'type': '물', 'power': 80}
{'pokemon': '이상해씨', 'type': '풀', 'power': 82}


In [27]:
# from_generator() 방법 - 메모리 효율적
def data_generator():
    """대용량 데이터에 유용한 제네레이터"""
    for i in range(10):
        yield {
            "id": i,
            "text": f"이것은 {i}번째 텍스트입니다.",
            "score": i * 0.1
        }

In [28]:
# Dataset 생성
gen_dataset = Dataset.from_generator(data_generator)
print("제네레이터에서 생성한 데이터셋:")
print(gen_dataset)
print("\n처음 3개 샘플:")
for i in range(3):
    print(gen_dataset[i])

Generating train split: 0 examples [00:00, ? examples/s]

제네레이터에서 생성한 데이터셋:
Dataset({
    features: ['id', 'text', 'score'],
    num_rows: 10
})

처음 3개 샘플:
{'id': 0, 'text': '이것은 0번째 텍스트입니다.', 'score': 0.0}
{'id': 1, 'text': '이것은 1번째 텍스트입니다.', 'score': 0.1}
{'id': 2, 'text': '이것은 2번째 텍스트입니다.', 'score': 0.2}


In [29]:
# IterableDataset 생성 (스트리밍용)
iterable_dataset = IterableDataset.from_generator(data_generator)
print("\n\n스트리밍 데이터셋:")
print(iterable_dataset)
print("스트리밍 샘플 (처음 3개):")
for i, sample in enumerate(iterable_dataset):
    if i >= 3:
        break
    print(sample)




스트리밍 데이터셋:
IterableDataset({
    features: ['id', 'text', 'score'],
    num_shards: 1
})
스트리밍 샘플 (처음 3개):
{'id': 0, 'text': '이것은 0번째 텍스트입니다.', 'score': 0.0}
{'id': 1, 'text': '이것은 1번째 텍스트입니다.', 'score': 0.1}
{'id': 2, 'text': '이것은 2번째 텍스트입니다.', 'score': 0.2}


## 4. 데이터셋 활용하기

로드한 데이터셋을 다양한 방법으로 조작하고 활용해봅시다.


In [30]:
# 데이터셋 기본 조작 예제
dataset = load_dataset("imdb", split="train[:1000]")  # 처음 1000개만 로드


### 4.1 데이터셋 정보


In [31]:
print("=== 데이터셋 정보 ===")
print(f"크기: {len(dataset)}")
print(f"컬럼: {dataset.column_names}")
print(f"특성: {dataset.features}")

=== 데이터셋 정보 ===
크기: 1000
컬럼: ['text', 'label']
특성: {'text': Value('string'), 'label': ClassLabel(names=['neg', 'pos'])}


### 4.2 인덱싱

In [32]:
# 인덱싱
print("\n=== 인덱싱 ===")
print("첫 번째 샘플:", dataset[0]["text"][:100] + "...")
print("처음 3개 라벨:", dataset[:3]["label"])


=== 인덱싱 ===
첫 번째 샘플: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it w...
처음 3개 라벨: [0, 0, 0]


### 4.3 슬라이싱

In [33]:
# 슬라이싱
print("\n=== 슬라이싱 ===")
subset = dataset[100:110]
print(f"100-110번째 샘플 개수: {len(subset['text'])}")


=== 슬라이싱 ===
100-110번째 샘플 개수: 10


### 4.4 셔플

In [34]:
# 셔플
print("\n=== 셔플 ===")
shuffled = dataset.shuffle(seed=42)
print("셔플 전 첫 번째 라벨:", dataset[0]["label"])
print("셔플 후 첫 번째 라벨:", shuffled[0]["label"])



=== 셔플 ===
셔플 전 첫 번째 라벨: 0
셔플 후 첫 번째 라벨: 0


### 4.5 필터링

In [35]:
# 필터링
print("=== 필터링 ===")
positive_reviews = dataset.filter(lambda example: example["label"] == 1)
print(f"긍정 리뷰 개수: {len(positive_reviews)}")

=== 필터링 ===


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

긍정 리뷰 개수: 0


### 4.6 정렬

In [37]:
# 정렬
print("\n=== 정렬 ===")
# text 길이를 저장할 새로운 컬럼 추가
dataset = dataset.map(lambda example: {"length": len(example["text"])})

# 'length' 컬럼 기준으로 오름차순 정렬
sorted_by_length = dataset.sort("length")

print("가장 짧은 리뷰:", sorted_by_length[0]["text"])
print("가장 긴 리뷰 길이:", len(sorted_by_length[-1]["text"]))


=== 정렬 ===


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

가장 짧은 리뷰: You'd better choose Paul Verhoeven's even if you have watched it.
가장 긴 리뷰 길이: 6103


### 4.7 선택

In [38]:
# 선택 (select)
print("\n=== 선택 ===")
sample_100 = dataset.select(range(100))
print(f"선택된 샘플 개수: {len(sample_100)}")


=== 선택 ===
선택된 샘플 개수: 100


### 4.8 분할

In [39]:
# 데이터셋 분할
print("\n=== 데이터셋 분할 ===")
train_test = dataset.train_test_split(test_size=0.2, seed=42)
print(f"훈련 데이터: {len(train_test['train'])}")
print(f"테스트 데이터: {len(train_test['test'])}")



=== 데이터셋 분할 ===
훈련 데이터: 800
테스트 데이터: 200


## 5. 데이터 전처리


### 5.1 map() 함수를 이용한 데이터 변환

`map()` 함수는 데이터셋의 모든 요소에 함수를 적용하는 가장 중요한 메서드입니다.

In [40]:
# 텍스트 전처리 함수 정의
def preprocess_text(examples):
    """텍스트를 소문자로 변환하고 길이 정보 추가"""
    return {
        "text_lower": [text.lower() for text in examples["text"]],
        "text_length": [len(text) for text in examples["text"]]
    }

In [41]:
# 작은 데이터셋으로 테스트
small_dataset = dataset.select(range(10))

# map 적용 (배치 처리)
processed = small_dataset.map(preprocess_text, batched=True)

print("=== 전처리 결과 ===")
for i in range(3):
    print(f"원본: {processed[i]['text'][:50]}...")
    print(f"소문자: {processed[i]['text_lower'][:50]}...")
    print(f"길이: {processed[i]['text_length']}")
    print("---")


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

=== 전처리 결과 ===
원본: I rented I AM CURIOUS-YELLOW from my video store b...
소문자: i rented i am curious-yellow from my video store b...
길이: 1640
---
원본: "I Am Curious: Yellow" is a risible and pretentiou...
소문자: "i am curious: yellow" is a risible and pretentiou...
길이: 1294
---
원본: If only to avoid making this type of film in the f...
소문자: if only to avoid making this type of film in the f...
길이: 528
---


### 5.2 토크나이저를 이용한 텍스트 토큰화


In [42]:
# 토크나이저 로드
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [43]:
def tokenize_function(examples):
    """텍스트를 토큰화하는 함수"""
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

In [44]:
# 토큰화 적용
tokenized_dataset = small_dataset.map(tokenize_function, batched=True)

print("=== 토큰화 결과 ===")
print("새로운 컬럼들:", tokenized_dataset.column_names)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

=== 토큰화 결과 ===
새로운 컬럼들: ['text', 'label', 'length', 'input_ids', 'token_type_ids', 'attention_mask']


In [45]:
# 첫 번째 샘플 확인
sample = tokenized_dataset[0]
print("원본 텍스트:", sample["text"][:100] + "...")
print("토큰 ID 개수:", len(sample["input_ids"]))
print("토큰 ID (처음 10개):", sample["input_ids"][:10])
print("어텐션 마스크 (처음 10개):", sample["attention_mask"][:10])


원본 텍스트: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it w...
토큰 ID 개수: 128
토큰 ID (처음 10개): [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026]
어텐션 마스크 (처음 10개): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### 5.3 PyTorch와 TensorFlow와의 호환성


In [46]:
# PyTorch 형식으로 변환
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [47]:
print("=== PyTorch 형식 ===")
print("데이터 형식:", tokenized_dataset.format)
sample = tokenized_dataset[0]
print("input_ids 타입:", type(sample["input_ids"]))
print("input_ids 형태:", sample["input_ids"].shape)


=== PyTorch 형식 ===
데이터 형식: {'type': 'torch', 'format_kwargs': {}, 'columns': ['input_ids', 'attention_mask', 'label'], 'output_all_columns': False}
input_ids 타입: <class 'torch.Tensor'>
input_ids 형태: torch.Size([128])


In [48]:
# 원래 형식으로 되돌리기
tokenized_dataset.reset_format()
print("\n=== 원래 형식으로 복원 ===")
print("데이터 형식:", tokenized_dataset.format)


=== 원래 형식으로 복원 ===
데이터 형식: {'type': None, 'format_kwargs': {}, 'columns': ['text', 'label', 'length', 'input_ids', 'token_type_ids', 'attention_mask'], 'output_all_columns': False}


In [49]:
print("\n=== 컬럼 조작 ===")
# 불필요한 컬럼 제거
clean_dataset = tokenized_dataset.remove_columns(["text"])
print("제거 후 컬럼:", clean_dataset.column_names)


=== 컬럼 조작 ===
제거 후 컬럼: ['label', 'length', 'input_ids', 'token_type_ids', 'attention_mask']


In [50]:
# 새 컬럼 추가
def add_ids(examples):
    return {"id": list(range(len(examples["label"])))}

dataset_with_ids = clean_dataset.map(add_ids, batched=True)
print("추가 후 컬럼:", dataset_with_ids.column_names)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

추가 후 컬럼: ['label', 'length', 'input_ids', 'token_type_ids', 'attention_mask', 'id']


### 5.4 스트리밍 데이터셋

대용량 데이터셋은 스트리밍 방식으로 처리하여 메모리 사용량을 줄일 수 있습니다.


In [51]:
# 스트리밍 데이터셋 로드
streaming_dataset = load_dataset("imdb", streaming=True)

print("=== 스트리밍 데이터셋 ===")
print("타입:", type(streaming_dataset["train"]))

=== 스트리밍 데이터셋 ===
타입: <class 'datasets.iterable_dataset.IterableDataset'>


In [52]:
# 스트리밍으로 데이터 처리
print("\n처음 5개 샘플 (스트리밍):")
for i, sample in enumerate(streaming_dataset["train"]):
    if i >= 5:
        break
    print(f"샘플 {i}: {sample['text'][:50]}... (라벨: {sample['label']})")


처음 5개 샘플 (스트리밍):
샘플 0: I rented I AM CURIOUS-YELLOW from my video store b... (라벨: 0)
샘플 1: "I Am Curious: Yellow" is a risible and pretentiou... (라벨: 0)
샘플 2: If only to avoid making this type of film in the f... (라벨: 0)
샘플 3: This film was probably inspired by Godard's Mascul... (라벨: 0)
샘플 4: Oh, brother...after hearing about this ridiculous ... (라벨: 0)


In [53]:
# 스트리밍 데이터셋에 map 적용
def simple_preprocess(example):
    example["text_length"] = len(example["text"])
    return example

processed_streaming = streaming_dataset["train"].map(simple_preprocess)

In [54]:
print("\n전처리된 스트리밍 데이터:")
for i, sample in enumerate(processed_streaming):
    if i >= 3:
        break
    print(f"길이: {sample['text_length']}, 텍스트: {sample['text'][:30]}...")



전처리된 스트리밍 데이터:
길이: 1640, 텍스트: I rented I AM CURIOUS-YELLOW f...
길이: 1294, 텍스트: "I Am Curious: Yellow" is a ri...
길이: 528, 텍스트: If only to avoid making this t...


## 6. 데이터셋 공유하기


### 6.1 로컬에 저장하는 방법

In [55]:
print("\n로컬 저장 방법:")
sample_dataset = Dataset.from_dict({
    "text": ["샘플 텍스트 1", "샘플 텍스트 2"],
    "label": [0, 1]
})


로컬 저장 방법:


In [56]:
# 로컬에 저장
save_path = os.path.join(temp_dir, "my_dataset")
sample_dataset.save_to_disk(save_path)
print(f"데이터셋 저장 완료: {save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

데이터셋 저장 완료: /tmp/tmpq4imftub/my_dataset


In [57]:
# 로컬에서 로드
loaded_dataset = Dataset.load_from_disk(save_path)
print("로드된 데이터셋:", loaded_dataset)
print("샘플:", loaded_dataset[0])


로드된 데이터셋: Dataset({
    features: ['text', 'label'],
    num_rows: 2
})
샘플: {'text': '샘플 텍스트 1', 'label': 0}


### 6.2 로컬에서 허브로 업로드

생성한 데이터셋을 Hugging Face Hub에 업로드하여 공유할 수 있습니다.

```python
# 1. 허브 로그인
from huggingface_hub import login
login()  # 또는 터미널에서 huggingface-cli login

# 2. 데이터셋 업로드
my_dataset = Dataset.from_dict({
    "text": ["안녕하세요", "반갑습니다", "감사합니다"],
    "label": [0, 1, 0]
})

# 허브에 업로드
my_dataset.push_to_hub("username/my-dataset")

# 여러 스플릿이 있는 경우
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
dataset_dict.push_to_hub("username/my-dataset")
```